## Analyse et intégration des données sur les objets perdus et la fréquentation SNCF, ainsi des données météorologiques de la ville de Paris

Ce notebook a comme base les données collectées en format CSV sur l'API SNCF concernant les objets perdus et la fréquentation, ainsi que des données météorologiques de la ville de Paris. Toutes ces données couvrent une période de 2021 à 2023. 

Nous allons nettoyer, agréger, normaliser ces données, ainsi que réaliser du feature engineering. Enfin, nous créerons une base de données SQLite3 pour rassembler l'ensemble de ces informations en un seul jeu de données.

## Dépendances

Les dépendances nécessaires à l’exécution du script sont les suivantes :
- **Pandas** : Utilisée pour la manipulation et l'analyse des données.
- **SQLite3** : Utilisée pour interagir avec les bases de données SQLite.
- **OS** : Utilisée pour gérer les chemins de fichiers et répertoires.

In [ ]:
import pandas as pd
import os
import sqlite3

# chargement des fichiers csv en noms de variables

In [1]:


meteo= pd.read_csv("../../csv_modélisé/all_meteo.csv")
objets= pd.read_csv("../../csv_modélisé/objets_trouves.csv")
frequentation= pd.read_csv("../../csv_modélisé/frequentation.csv")

# Enlever éventuels doublons dans les données

In [ ]:
meteo=meteo.drop_duplicates()
frequentation=frequentation.drop_duplicates()
objets=objets.drop_duplicates()

# mappage de colonne pour avoir le même nom de gare entre les tables

In [2]:
frequentation['gare'] = frequentation['gare'].replace(
    {"Paris Bercy Bourgogne - Pays d'Auvergne": "Paris Bercy"}
)

# Arrondir les valeures des données météo "cloudcover" et "sunhour" , et conversion type en INT

In [6]:



# Arrondir les valeurs des colonnes sélectionnées
columns_to_round = ['CLOUDCOVER_AVG_PERCENT', 'SUNHOUR']
meteo[columns_to_round] = meteo[columns_to_round].round().astype(int)



# Reconstruction du dataframe "météo" à partir des colonnes de météo strictmenet nécessaires (principe de minimisation DES données RGPD)

In [7]:
meteo = meteo[['DATE'] + columns_to_round]

# visualisation des colonnes actuelles du dataframe météo

In [ ]:
meteo.columns

# garder 10 premiers caractéres des lignes date pour un format commun

In [8]:
objets['date'] = objets['date'].str[:10]
# va permettre d'avoir un format de date commun à la table météo
objets.head(5)


,date,type,gare
0,2021-01-01,"Bagagerie: sacs, valises, cartables",Paris Gare de Lyon
1,2021-01-02,"Bagagerie: sacs, valises, cartables",Paris Gare de Lyon
2,2021-01-02,"Bagagerie: sacs, valises, cartables",Paris Gare de Lyon
3,2021-01-02,"Porte-monnaie / portefeuille, argent, titres",Paris Gare de Lyon
4,2021-01-02,"Bagagerie: sacs, valises, cartables",Paris Gare de Lyon


# mappage des valeures du champ type pour faciliter analyse

In [9]:
mappage = {
    'Bagagerie: sacs, valises, cartables': 'bagages',
    'Porte-monnaie / portefeuille, argent, titres': 'clé',
    'Appareils électroniques, informatiques, appareils photo': 'electronique',
    'Articles médicaux': 'médical',
    'Pièces d identités et papiers personnels': 'papiers',
    'Clés, porte-clés, badge magnétique': 'clé',
    'Optique': 'optique',
    'Divers': 'divers',
    'Articles d enfants, de puériculture': 'articles enfants',
    'Vêtements, chaussures': 'vetements',
    'Vélos, trottinettes, accessoires 2 roues': 'velo/trottinette',
    'Instruments de musique': 'article musique',
    'Bijoux, montres': 'bijoux/montres',
    'Articles de sport, loisirs, camping': 'sport/loisirs',
    'Livres, articles de papéterie': 'livres',
    'Parapluies': 'parapluies'
}

# Appliquer le mappage à la colonne 'type'
objets['type'] = objets['type'].map(mappage)

# Afficher les premières lignes du DataFrame mis à jour pour vérification
print(objets.head())



         date     type                gare
0  2021-01-01  bagages  Paris Gare de Lyon
1  2021-01-02  bagages  Paris Gare de Lyon
2  2021-01-02  bagages  Paris Gare de Lyon
3  2021-01-02      clé  Paris Gare de Lyon
4  2021-01-02  bagages  Paris Gare de Lyon


# visualisation des différentes valeures du champ "type"

In [10]:
valeurs_uniques = objets['type'].unique()

# Afficher les valeurs uniques
print("Valeurs uniques dans la colonne 'type' :")
print(valeurs_uniques)

Valeurs uniques dans la colonne 'type' :
['bagages' 'clé' 'electronique' 'médical' 'papiers' 'optique' 'divers'
 'articles enfants' 'vetements' 'velo/trottinette' 'article musique'
 'bijoux/montres' 'sport/loisirs' 'livres' 'parapluies']


# entregistrement partiel des résultats en format csv pour éviter d'importer à nouveau toutes les données 

In [11]:
meteo.to_csv('../../csv_modélisé/all_meteo.csv', index=False)
objets.to_csv('../../csv_modélisé/objets_trouves_bis.csv', index=False)
frequentation.to_csv('../../csv_modélisé/frequentation.csv', index=False)

# création de bdd sqlite "bdd_pondere.db"

In [13]:
import os
import sqlite3

# Obtenir le répertoire du script actuel
repertoire_actuel = os.getcwd()

# Définir le chemin pour la base de données (dans le répertoire actuel)
chemin_bdd = os.path.join(repertoire_actuel, 'bdd_pondere.db')

# Connexion à la base de données SQLite
connexion = sqlite3.connect(chemin_bdd)
curseur = connexion.cursor()

# Supprimer les tables existantes si elles existent
curseur.execute("DROP TABLE IF EXISTS frequentation")
curseur.execute("DROP TABLE IF EXISTS lumiere")
curseur.execute("DROP TABLE IF EXISTS objets_trouves")

# Créer la table 'frequentation' avec nom_gare comme clé primaire
curseur.execute("""
    CREATE TABLE frequentation(
        nom_gare TEXT PRIMARY KEY NOT NULL,
        frequent_2021 INTEGER,
        frequent_2022 INTEGER,
        frequent_2023 INTEGER
    )
""")

# Créer la table 'lumiere'
curseur.execute("""
    CREATE TABLE lumiere(
        date DATE NOT NULL PRIMARY KEY,
        cloud INTEGER,
        sun INTEGER,
        annee INTEGER 
    )
""")

# Créer la table 'objets_trouves' avec une clé étrangère pour la gare
curseur.execute("""
    CREATE TABLE objets_trouves(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE NOT NULL,
        type TEXT,
        nom_gare TEXT NOT NULL,
        poids_pondere FLOAT, 
        FOREIGN KEY (date) REFERENCES lumiere(date),
        FOREIGN KEY (nom_gare) REFERENCES frequentation(nom_gare)
    )
""")

# Fermer la connexion
connexion.commit()
connexion.close()

print(f"Base de données créée dans : {chemin_bdd}")

Base de données créée dans : /home/utilisateur/Documents/dernier_bdd/certification_finale_bdd_E1/lancement_scripts_projet/nettoyage_des_donnees/bdd_pondere.db


# création de bdd sqlite "luminosite.db"

In [14]:
repertoire_actuel = os.getcwd()

# Définir le chemin pour la base de données (dans le répertoire actuel)
chemin_bdd = os.path.join(repertoire_actuel, 'bdd_luminosite.db')

# Connexion à la base de données SQLite
connexion = sqlite3.connect(chemin_bdd)
curseur = connexion.cursor()

# Supprimer les tables existantes si elles existent
curseur.execute("DROP TABLE IF EXISTS frequentation")
curseur.execute("DROP TABLE IF EXISTS lumiere")
curseur.execute("DROP TABLE IF EXISTS objets_trouves")

# Créer la table 'frequentation' avec nom_gare comme clé primaire
curseur.execute("""
    CREATE TABLE frequentation(
        nom_gare TEXT PRIMARY KEY NOT NULL,
        frequent_2021 INTEGER,
        frequent_2022 INTEGER,
        frequent_2023 INTEGER
    )
""")

# Créer la table 'lumiere'
curseur.execute("""
    CREATE TABLE lumiere(
        date TEXT NOT NULL PRIMARY KEY,
        cloud INTEGER,
        sun INTEGER,
        annee INTEGER 
    )
""")

# Créer la table 'objets_trouves' avec une clé étrangère pour la gare
curseur.execute("""
    CREATE TABLE objets_trouves(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE NOT NULL,
        type TEXT,
        nom_gare TEXT NOT NULL,
        poids_pondere FLOAT,
        FOREIGN KEY (date) REFERENCES lumiere(date),
        FOREIGN KEY (nom_gare) REFERENCES frequentation(nom_gare)
    )
""")

# Fermer la connexion
connexion.commit()
connexion.close()

print(f"Base de données créée dans : {chemin_bdd}")


Base de données créée dans : /home/utilisateur/Documents/dernier_bdd/certification_finale_bdd_E1/lancement_scripts_projet/nettoyage_des_donnees/bdd_luminosite.db


# import des données dans base de données sqlite (à partir des différents dataframes)

In [15]:
import sqlite3

# Connexion à la première base de données
conn_luminosite = sqlite3.connect('bdd_luminosite.db')
curseur_luminosite = conn_luminosite.cursor()

# Connexion à la seconde base de données
conn_pondere = sqlite3.connect('bdd_pondere.db')
curseur_pondere = conn_pondere.cursor()



# Créer les tables dans la base de données bdd_pondere.db


# Insérer les données dans la table 'lumiere'
for index, row in meteo.iterrows():
    curseur_luminosite.execute("INSERT INTO lumiere (date, cloud, sun) VALUES (?, ?, ?)", 
                               (row['DATE'], row['CLOUDCOVER_AVG_PERCENT'], row['SUNHOUR']))
    curseur_pondere.execute("INSERT INTO lumiere (date, cloud, sun) VALUES (?, ?, ?)", 
                            (row['DATE'], row['CLOUDCOVER_AVG_PERCENT'], row['SUNHOUR']))

# Insérer les données dans la table 'frequentation'
for index, row in frequentation.iterrows():
    curseur_luminosite.execute("INSERT INTO frequentation (nom_gare, frequent_2021, frequent_2022, frequent_2023) VALUES (?, ?, ?, ?)", 
                               (row['gare'], row['frequent_2021'], row['frequent_2022'], row['frequent_2023']))
    curseur_pondere.execute("INSERT INTO frequentation (nom_gare, frequent_2021, frequent_2022, frequent_2023) VALUES (?, ?, ?, ?)", 
                            (row['gare'], row['frequent_2021'], row['frequent_2022'], row['frequent_2023']))

# Insérer les données dans la table 'objets_trouves'
for index, row in objets.iterrows():
    curseur_luminosite.execute("INSERT INTO objets_trouves (date, type, nom_gare) VALUES (?, ?, ?)", 
                               (row['date'], row['type'], row['gare']))
    curseur_pondere.execute("INSERT INTO objets_trouves (date, type, nom_gare) VALUES (?, ?, ?)", 
                            (row['date'], row['type'], row['gare']))

# Valider toutes les transactions en une fois
conn_luminosite.commit()
conn_pondere.commit()

# Fermer les connexions
conn_luminosite.close()
conn_pondere.close()

print("Données insérées avec succès dans les deux bases de données.")

Données insérées avec succès dans les deux bases de données.


# Normalisation de données: création d'un dataframe qui reprend les données de la table objets trouvés, calcul et ajout de la valeur du poids pondéré à ce datafarame

## Calcul du champ "poids pondéré" 
(sera un indice de valeur de chaque ligne d'objet trouvé, sur lequel on pourra s'appuyer pour l'analyse)

1/. Création de 2 DataFrames à partir des tables `frequentation` et `objets_trouves` de la base de données `bdd_luminosité.db`.

  
2/. Création d'un champ "année" à partir du champ "date" pour le DataFrame `df_objets`.


3/. Initialisation d'un champ "poids_pondere" à `df_objets`.


4/. Parcours de toutes les lignes de `df_objets`. À partir de la correspondance des champs "année" et "gare" avec les valeurs du DataFrame `df_frequentation`, on récupère la valeur de fréquentation pour l'insérer dans le champ "poids_pondere" de la même ligne.


5/. Vérification du mappage correct avec l'affichage des lignes de `df_objets`.


In [16]:
import os
import sqlite3
import pandas as pd

# Obtenir le répertoire du script actuel
repertoire_script = os.getcwd()

# Définir les chemins pour les bases de données
chemin_bdd_source = os.path.join(repertoire_script, 'bdd_luminosite.db')

# Connexion à la base de données source
conn_source = sqlite3.connect(chemin_bdd_source)


df_frequentation = pd.read_sql_query("SELECT * FROM frequentation", conn_source)                                # Charge les données de la table 'frequentation' dans un DataFrame

df_objets = pd.read_sql_query("SELECT * FROM objets_trouves", conn_source)                                      # Charge les données de la table 'objets_trouves' dans un DataFrame                         


# Fermer la connexion après la lecture
conn_source.close()

                                                                                                                # Étape 1 : Convertir la colonne 'date' en datetime et ajouter une colonne 'année'
df_objets['date'] = pd.to_datetime(df_objets['date'])  # Convertir en datetime
df_objets['année'] = df_objets['date'].dt.year  # Extraire l'année

                                                                                                                # Étape 2 : Calculer la fréquentation minimale de 2021
frequent_min_2021 = df_frequentation['frequent_2021'].min()

                                                                                                                # Étape 3 et 4 : Remplir la colonne 'poids_pondere'
df_objets['poids_pondere'] = 0.0                                                                                # Initialisation à 0

                                                                                                                # Calcule le poids pondéré pour chaque objet trouvé
for index, row in df_objets.iterrows():
    annee = row['année']                                                                                        # Récupére l'année du df des objets trouvés
    nom_gare = row['nom_gare']                                                                                  # Récupére le nom de la gare du df des objets trouvés

    
    variable_a = df_frequentation.loc[df_frequentation['nom_gare'] == nom_gare, f'frequent_{annee}'].values     # calcul de la fréquentation relative à l'objet trouvé (selon sa gare et année de perte)

    if variable_a.size > 0:                                                                                     # Vérifie si la valeur existe
        variable_a = variable_a[0]                                                                              # Prendre la première valeur

        
        df_objets.at[index, 'poids_pondere'] = frequent_min_2021 / variable_a                                   # Calcule le poids pondéré

                                                                                                                # Affiche les résultats
print(df_objets[['date', 'type', 'nom_gare', 'année', 'poids_pondere']])


            date          type            nom_gare  année  poids_pondere
0     2021-01-01       bagages  Paris Gare de Lyon   2021       0.036069
1     2021-01-02       bagages  Paris Gare de Lyon   2021       0.036069
2     2021-01-02       bagages  Paris Gare de Lyon   2021       0.036069
3     2021-01-02           clé  Paris Gare de Lyon   2021       0.036069
4     2021-01-02       bagages  Paris Gare de Lyon   2021       0.036069
...          ...           ...                 ...    ...            ...
46321 2023-12-24       bagages         Paris Bercy   2023       0.659723
46322 2023-12-24       bagages         Paris Bercy   2023       0.659723
46323 2023-12-25       bagages         Paris Bercy   2023       0.659723
46324 2023-12-27           clé         Paris Bercy   2023       0.659723
46325 2023-12-31  electronique         Paris Bercy   2023       0.659723

[46326 rows x 5 columns]


# insertion des données avec le poids pondéré depuis le dataframe créé à la base de données sqlite

In [17]:
repertoire_actuel = os.getcwd()

# Définir le chemin pour la base de données
chemin_bdd = os.path.join(repertoire_actuel, 'bdd_pondere.db')

# Connexion à la base de données SQLite
connexion = sqlite3.connect(chemin_bdd)
curseur = connexion.cursor()

# Supprimer les anciennes données de la table objets_trouves (facultatif)
curseur.execute("DELETE FROM objets_trouves")

# Insérer les nouvelles données du DataFrame dans la table objets_trouves
for index, row in df_objets.iterrows():
    curseur.execute("""
        INSERT INTO objets_trouves (date, type, nom_gare, poids_pondere)
        VALUES (?, ?, ?, ?)
    """, (row['date'].strftime('%Y-%m-%d'), row['type'], row['nom_gare'], row['poids_pondere']))

# Valider les changements et fermer la connexion
connexion.commit()
connexion.close()

print("Les données de la table objets_trouves ont été remplacées avec succès.")

Les données de la table objets_trouves ont été remplacées avec succès.


# visuel de la base donnée sqlite "bdd_pondere"

In [18]:
import sqlite3
import os

# Obtenir le répertoire actuel
repertoire_actuel = os.getcwd()

# Définir le chemin pour la base de données
chemin_bdd = os.path.join(repertoire_actuel, 'bdd_pondere.db')

# Connexion à la base de données SQLite
connexion = sqlite3.connect(chemin_bdd)
curseur = connexion.cursor()

# Afficher les premières lignes de la table objets_trouves
print("Premières lignes de la table 'objets_trouves':")
curseur.execute("SELECT * FROM objets_trouves LIMIT 5")
rows = curseur.fetchall()
for row in rows:
    print(row)

# Afficher les types des colonnes
print("\nTypes des colonnes de la table 'objets_trouves':")
curseur.execute("PRAGMA table_info(objets_trouves)")
columns = curseur.fetchall()
for column in columns:
    print(f"Colonne: {column[1]}, Type: {column[2]}")

# Fermer la connexion
connexion.close()


Premières lignes de la table 'objets_trouves':
(46327, '2021-01-01', 'bagages', 'Paris Gare de Lyon', 0.036069450579702)
(46328, '2021-01-02', 'bagages', 'Paris Gare de Lyon', 0.036069450579702)
(46329, '2021-01-02', 'bagages', 'Paris Gare de Lyon', 0.036069450579702)
(46330, '2021-01-02', 'clé', 'Paris Gare de Lyon', 0.036069450579702)
(46331, '2021-01-02', 'bagages', 'Paris Gare de Lyon', 0.036069450579702)

Types des colonnes de la table 'objets_trouves':
Colonne: id, Type: INTEGER
Colonne: date, Type: DATE
Colonne: type, Type: TEXT
Colonne: nom_gare, Type: TEXT
Colonne: poids_pondere, Type: FLOAT
